In [7]:
import transformers

m = transformers.BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
m

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [9]:
lm_head = m.cls

In [10]:
lm_head

BertOnlyMLMHead(
  (predictions): BertLMPredictionHead(
    (transform): BertPredictionHeadTransform(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (transform_act_fn): GELUActivation()
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    )
    (decoder): Linear(in_features=768, out_features=30522, bias=True)
  )
)

### Understand how the LM candidate substitution generation works

In [3]:
def onwards_token_predict(averaged_activation, layer_id, model, flip_target, gpu_device_num):
    final_hidden = model(inputs_embeds=averaged_activation, output_hidden_states=True).hidden_states[-1]
    predictions = model.lm_head(final_hidden)
    return predictions

def get_embeddings(model, ids, gpu_device_num):
    if isRoberta(model):
        word_embedding = model.roberta.embeddings(ids)
    elif isBert(model):
        token_type_ids = get_token_type_ids(ids).cuda(gpu_device_num)
        #token_type_embeddings = model.bert.embeddings.token_type_embeddings(token_type_ids)
        word_embedding = model.bert.embeddings(input_ids=ids, token_type_ids=token_type_ids)
    else:
        print("Error: unknown model type:", type(model))
    return word_embedding

hidden_state = get_embeddings(LM_model, ids, gpu_device_num).cuda(gpu_device_num)
sample_logits = onwards_token_predict(candidate_hidden_state, layer, LM_model, forward_prediction_target, gpu_device_num)[0]


NameError: name 'LM_model' is not defined

In [23]:
from transformers import AutoTokenizer

s = "The quick brown fox jumped over the lazy dog."
print(s.split(), len(s.split()))
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
ids = tokenizer.encode(s, return_tensors="pt")
print(ids, ids.size())

['The', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog.'] 9
tensor([[  101,  1996,  4248,  2829,  4419,  5598,  2058,  1996, 13971,  3899,
          1012,   102]]) torch.Size([1, 12])


In [26]:
tokens = tokenizer.convert_ids_to_tokens(ids[0])
print(tokens, len(tokens))

['[CLS]', 'the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', '.', '[SEP]'] 12


Get the input embeddings for the tokens.

In [17]:
word_embedding = m.bert.embeddings(input_ids=ids)
word_embedding.shape

torch.Size([1, 12, 768])

Run the embedding through the model to get the encoded state and then use this to get the logit predictions from the LM head.

In [35]:
final_hidden = m(inputs_embeds=word_embedding, output_hidden_states=True).hidden_states[-1]
final_hidden.shape

torch.Size([1, 12, 768])

In [40]:
predictions = m.cls(final_hidden)
predictions.size()

torch.Size([1, 12, 30522])

## Understand the onwards token predict function

In [8]:
import transformers
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
LM_model = transformers.BertForMaskedLM.from_pretrained("bert-base-uncased").to(device)
LM_model.lm_head = LM_model.cls

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")

In [11]:
sentence = "The quick brown fox."
tokens = tokenizer.tokenize(sentence)
tokens

['the', 'quick', 'brown', 'fox', '.']

In [12]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[1996, 4248, 2829, 4419, 1012]

In [13]:
input_tensor = torch.tensor(ids).to(device).view(1, -1)
input_tensor.shape

torch.Size([1, 5])

In [15]:
def get_token_type_ids(tokens):
    passed_SEP = False
    if type(tokens) == torch.Tensor:
        tokens = tokens.view(-1).tolist()
    token_type_ids = [0 for _ in range(len(tokens))]
    #print(tokens)
    if type(tokens[0]) == str:
        for i in range(len(tokens)):
            if passed_SEP:
                token_type_ids[i] = 1
            if tokens[i] == '[SEP]':
                passed_SEP = True
    else:
        for i in range(len(tokens)):
            if passed_SEP:
                token_type_ids[i] = 1
            if tokens[i] == 102:
                passed_SEP = True
    return torch.tensor([token_type_ids])

def get_embeddings(model, ids, device):
    token_type_ids = get_token_type_ids(ids).to(device)
    word_embedding = model.bert.embeddings(input_ids=ids, token_type_ids=token_type_ids)
    return word_embedding

hidden_state = get_embeddings(LM_model, ids, device).to(device)
hidden_state.shape

NameError: name 'get_token_type_ids' is not defined

In [2]:
def onwards_token_predict(averaged_activation, layer_id, model):
    final_hidden = model(inputs_embeds=averaged_activation, output_hidden_states=True).hidden_states[-1]
    predictions = model.lm_head(final_hidden)
    return predictions